In [1]:
import numpy as np
import em_algorithm
from simulation_framework.simulate_competency import respondent_population
from simulation_framework.simulate_responses import response_simulation
from scipy.stats import multivariate_normal
import models
import em_algorithm
import pandas as pd
import time
from girth import multidimensional_twopl_mml
from girth import twopl_mml
import cma
import scipy

C:\Users\Jesper\Documents\GitHub\Knowledge-Growth-Prediction\models


In [2]:
def params_to_vector(model):
    item_dimension = model.item_dimension
    latent_dimension = model.latent_dimension
    A_flat = model.item_parameters["discrimination_matrix"].flatten()
    A_flat = A_flat[A_flat != 0]
    A_cut = len(A_flat)
    delta_cut = A_cut+item_dimension
    delta = model.item_parameters["intercept_vector"]
    sigma_flat = model.person_parameters["covariance"][np.triu_indices(
            model.latent_dimension, k=1)]
    return(np.concatenate((A_flat, delta, sigma_flat), axis=0))

def vector_to_params(vector, model):
    item_dimension = model.item_dimension
    latent_dimension = model.latent_dimension 
    q_matrix = model.item_parameters["q_matrix"] 
    q_flat = q_matrix.flatten()
    A_cut = len(q_flat[q_flat != 0])
    delta_cut = A_cut+item_dimension
    A = vector[0:A_cut]
    A = model.fill_zero_discriminations(A).reshape((item_dimension, latent_dimension))
    delta = vector[A_cut:delta_cut]
    item_parameters = {"discrimination_matrix": A, "intercept_vector": delta}
    corr = vector[delta_cut: len(vector)]
    sigma = model.corr_to_sigma(corr, False)
    return({"item_parameters": item_parameters, "person_parameters": {"covariance": sigma}})

def direct_marginal_optimization(model, response_data):
    #Get initial parameters
    x0 = params_to_vector(model)
    response_data = response_data.to_numpy()
    #Optimize with GA
    def marginal_l_func(input_vector):
        parameters = vector_to_params(input_vector, model)
        try:
            model.set_parameters(parameters)
        except Exception:
            return(np.inf)
        result = -1*model.marginal_response_loglikelihood(response_data)
        return(result)
    es = cma.CMAEvolutionStrategy(x0=x0, sigma0=0.5)
    es.optimize(marginal_l_func, maxfun=100000)
    #Get result
    result = es.result.xfavorite
    return(vector_to_params(result, model))

In [3]:
from models.mirt_2pl import mirt_2pl


def rmse(y_pred: np.array, y_true: np.array) -> float:
    MSE = np.square(np.subtract(y_pred.flatten(), y_true.flatten())).mean()
    RMSE = np.sqrt(MSE)
    return(float(RMSE))

def experiment_performance(estimated_parameter_dict, real_parameter_dict):
    res_dict = {}
    if "item_parameters" in estimated_parameter_dict.keys():
        A_pred = estimated_parameter_dict["item_parameters"]["discrimination_matrix"]
        delta_pred = estimated_parameter_dict["item_parameters"]["intercept_vector"]

        A_true = real_parameter_dict["item_parameters"]["discrimination_matrix"]
        delta_true = real_parameter_dict["item_parameters"]["intercept_vector"]

        print("Absolute diff in A:")
        print(np.abs(A_true-A_pred))

        print("Absolute diff in delta:")
        print(np.abs(delta_true-delta_pred))

        res_dict["rmse_A"] = rmse(A_pred, A_true)
        res_dict["rmse_delta"] = rmse(delta_true, delta_pred)

    if "person_parameters" in estimated_parameter_dict.keys():
        sigma_pred = estimated_parameter_dict["person_parameters"]["covariance"]
        sigma_true = real_parameter_dict["person_parameters"]["covariance"]

        print("Absolute diff in sigma:")
        print(np.abs(sigma_true-sigma_pred)) 

        res_dict["rmse_sigma"] = rmse(sigma_true, sigma_pred) 
    if len(res_dict.keys())==0:
        raise Exception("No performance to calculate")
    return(res_dict)

def standardize_parameters(parameter_dict):
        covariance = parameter_dict["person_parameters"]["covariance"]
        A = parameter_dict["item_parameters"]["discrimination_matrix"]
        if len(covariance.shape)<=1:
            standardized_cov = np.array([[1]])
            inv_sqrt_cov = np.array([[np.sqrt(1/covariance)]])
            A = np.multiply(A, inv_sqrt_cov)
        else:
            inv_sqrt_cov = np.linalg.inv(scipy.linalg.sqrtm(covariance))
            standardized_cov = np.dot(np.dot(inv_sqrt_cov, covariance), inv_sqrt_cov)
            A = np.dot(A, np.linalg.inv(inv_sqrt_cov))
        parameter_dict["item_parameters"]["discrimination_matrix"] = A
        parameter_dict["person_parameters"]["covariance"] = standardized_cov
        return(parameter_dict)

def create_parameter_dict(estimated_early_parameters, real_early_parameters, estimated_late_parameters, real_late_parameters):
    parameter_dict = {"real_early_parameters": real_early_parameters,
                   "estimated_early_parameters": estimated_early_parameters}
    return(parameter_dict)


def create_performance_dict(parameter_dict, run_dict, sample=None, baselines=None, model=None):
    result_dict = parameter_dict
    result_dict["sample"] = sample
    #Model Performance
    result_dict["early_performance"] = {}
    result_dict["early_performance"]["rmse"] = experiment_performance(real_parameter_dict=result_dict["real_early_parameters"], 
                                                              estimated_parameter_dict=result_dict["estimated_early_parameters"])
    #Baseline's Performance
    baselines["early_initial"]["performance"] = {"rmse": experiment_performance(real_parameter_dict=result_dict["real_early_parameters"], 
                                                              estimated_parameter_dict=baselines["early_initial"]["parameters"])}
    if "girth" in baselines.keys():
        girth_data = result_dict["sample"]["early_responses"].to_numpy().transpose()
        if model.latent_dimension == 1:
            girth_estimates = twopl_mml(girth_data)
        else:
            girth_estimates = multidimensional_twopl_mml(girth_data, n_factors=model.latent_dimension)
        girth_item_parameters = {"discrimination_matrix": girth_estimates["Discrimination"], "intercept_vector": girth_estimates["Difficulty"]}
        girth_parameters = {"item_parameters": girth_item_parameters}
        girth_estimated_covariance = np.cov(girth_estimates["Ability"], rowvar=True)
        girth_parameters["person_parameters"] = {"covariance": girth_estimated_covariance}
        girth_parameters = standardize_parameters(girth_parameters)
        girth_performance_rmse = experiment_performance(real_parameter_dict=result_dict["real_early_parameters"], 
                                                              estimated_parameter_dict=girth_parameters)
        baselines["girth"]["parameters"] = girth_parameters
        baselines["girth"]["performance"] = {"rmse": girth_performance_rmse}

    if "early_direct" in baselines.keys():
        dir_model = mirt_2pl(item_dimension=model.item_dimension, latent_dimension=model.latent_dimension, Q=model.item_parameters["q_matrix"])
        direct_early_item_parameters = direct_marginal_optimization(dir_model, response_data=sample["early_responses"])
        direct_early_performance_rmse = experiment_performance(real_parameter_dict=result_dict["real_early_parameters"], 
                                                              estimated_parameter_dict=direct_early_item_parameters)
        baselines["early_direct"]["parameters"] = direct_early_item_parameters
        baselines["early_direct"]["performance"] = {"rmse": direct_early_performance_rmse}

    result_dict["baselines"] = baselines
    likelihood = calculate_marginal_likelihoods(model=model, response_data=sample["early_responses"], real_parameters=result_dict["real_early_parameters"],
                                                initial_parameters=baselines["early_initial"]["parameters"], estimated_parameters=result_dict["estimated_early_parameters"])
    result_dict["early_performance"]["marginal_likelihood"] = likelihood
    result_dict["early_performance"]["run"] = run_dict["early"]
    return(result_dict)

def calculate_marginal_likelihoods(model, response_data, real_parameters, initial_parameters, estimated_parameters):
    model.set_parameters(initial_parameters)
    initial_marginal_likelihood = model.marginal_response_loglikelihood(response_data=response_data.to_numpy())
    model.set_parameters(real_parameters)
    optimal_marginal_likelihood = model.marginal_response_loglikelihood(response_data=response_data.to_numpy())
    model.set_parameters(estimated_parameters)
    marginal_likelihood_estimated = model.marginal_response_loglikelihood(response_data=response_data.to_numpy())
    likelihood_dict = {"optimal": optimal_marginal_likelihood, "estimated": marginal_likelihood_estimated, "initial": initial_marginal_likelihood}
    return(likelihood_dict)

In [4]:
def direct_optimization_benchmark():
    latent_dimension = 2
    item_dimension = 6
    sample_size=30

    #Define Population
    real_latent_cov = np.array([[1,0.2],
                    [0.2,1]])

    latent_distribution = multivariate_normal(mean=np.array([0,0]), cov=real_latent_cov)
    population = respondent_population(latent_dimension=latent_dimension, latent_distribution=latent_distribution)

    #Define Test
    A = np.array([[0.5,0],
                  [2,0],
                  [0,0.4],
                  [0.7,0.6],
                  [2, 0.9],
                  [0, 1.5]])
    Q = np.array([[1,0],
                  [1,0],
                  [0,1],
                  [1,1],
                  [1,1],
                  [0,1]])

    delta = np.array([0, 0.5, 1, 0, 1.5, 0.9])
    early_item_parameters = {"discrimination_matrix": A, "intercept_vector": delta, "q_matrix": Q, "item_dimension": item_dimension, "latent_dimension": latent_dimension}
    real_early_parameters = {"item_parameters": early_item_parameters, "person_parameters": {"covariance": real_latent_cov}}

    #Sample responses
    response_simulation_obj = response_simulation(population=population, item_dimension = item_dimension, early_item_params=early_item_parameters)
    sample = response_simulation_obj.sample(100)

    #Fit Parameters
    #Initialize model
    model = models.mirt_2pl(latent_dimension=2, item_dimension=6, Q=Q)
    model.initialize_from_responses(response_data=sample["early_responses"])

    estimated_params = direct_marginal_optimization(model, response_data=sample["early_responses"])
    return(estimated_params)

In [5]:
#direct_result_dict = direct_optimization_benchmark()

## Experiment 0: MIRT-2PL Performance Benchmark

In [6]:
def mirt_performance_benchmark() -> dict:
    latent_dimension = 2
    item_dimension = 6
    sample_size=30

    #Define Population
    real_latent_cov = np.array([[1,0.2],
                    [0.2,1]])

    latent_distribution = multivariate_normal(mean=np.array([0,0]), cov=real_latent_cov)
    population = respondent_population(latent_dimension=latent_dimension, latent_distribution=latent_distribution)

    #Define Test
    A = np.array([[0.5,0],
                  [2,0],
                  [0,0.4],
                  [0.7,0.6],
                  [2, 0.9],
                  [0, 1.5]])
    Q = np.array([[1,0],
                  [1,0],
                  [0,1],
                  [1,1],
                  [1,1],
                  [0,1]])

    delta = np.array([0, 0.5, 1, 0, 1.5, 0.9])
    early_item_parameters = {"discrimination_matrix": A, "intercept_vector": delta, "q_matrix": Q, "item_dimension": item_dimension, "latent_dimension": latent_dimension}
    real_early_parameters = {"item_parameters": early_item_parameters, "person_parameters": {"covariance": real_latent_cov}}

    #Sample responses
    response_simulation_obj = response_simulation(population=population, item_dimension = item_dimension, early_item_params=early_item_parameters)
    sample = response_simulation_obj.sample(100)

    #Fit Parameters
    #Initialize model
    model = models.mirt_2pl(latent_dimension=2, item_dimension=6, Q=Q)
    model.initialize_from_responses(response_data=sample["early_responses"])
    initial_early_parameters = model.get_parameters()
    e_step = em_algorithm.e_step_ga_mml(model=model)
    m_step = em_algorithm.m_step_ga_mml(model)
    em = em_algorithm.em_algo(e_step=e_step, m_step=m_step, model=model)

    #Fit Model
    start_time = time.time()
    em.fit(sample["early_responses"], max_iter=50)
    run_time =  (time.time() - start_time)

    #Create Baselines
    baselines = {"early_initial": {"parameters": initial_early_parameters}, "girth": {}, "early_direct": {}}
    
    #Create results
    early_estimated_parameters = em.model.get_parameters()

    parameter_dict = create_parameter_dict(estimated_early_parameters=early_estimated_parameters,
                          real_early_parameters = real_early_parameters,
                          estimated_late_parameters=None, real_late_parameters=None)
    run_dict = {"early": {"runtime": run_time,
                "number_steps": em.n_steps}}
    performance_dict = create_performance_dict(parameter_dict=parameter_dict, run_dict=run_dict, sample=sample, baselines=baselines, model=model)
    return(performance_dict)

In [18]:

def print_result(result_dict, description=""):
    #Description
    print("------------------------------------")
    print("##### Results for {0}".format(description))
    #Performance/time
    ep_dict = result_dict["early_performance"]
    runtime = np.round(ep_dict["run"]["runtime"], 2)
    steps = ep_dict["run"]["number_steps"]
    print("Runtime: {0} seconds, {1} steps, {2} seconds per step \\".format(runtime, steps, np.round(runtime/steps, 2)))
    #Performance/results
    l_optimal = np.round(ep_dict["marginal_likelihood"]["optimal"], 2)
    l_estimated = np.round(ep_dict["marginal_likelihood"]["estimated"], 2)
    l_real = np.round(ep_dict["marginal_likelihood"]["initial"], 2)
    print("Optimal marginal Likelihood: {0}, Estimated: {1}, Initial {2}".format(l_optimal,l_estimated,l_real))
    #print("Performance: rmse-mean = {0} \\".format(np.round(np.mean(np.array(list(result_dict["early_performance"].values()))), 4)))
    rmse_model = ep_dict["rmse"]
    rmse_girth = result_dict["baselines"]["girth"]["performance"]["rmse"]
    rmse_initial = result_dict["baselines"]["early_initial"]["performance"]["rmse"]
    
    rmse_frame = pd.DataFrame(columns=["rmse_A", "rmse_delta", "rmse_sigma"])
    rmse_frame = rmse_frame.append(rmse_model, ignore_index=True)
    rmse_frame = rmse_frame.append(rmse_girth, ignore_index=True)
    
    rmse_frame = rmse_frame.append(rmse_initial, ignore_index=True)
    if "early_direct" in result_dict["baselines"].keys():
        rmse_direct = result_dict["baselines"]["early_direct"]["performance"]["rmse"]
        rmse_frame = rmse_frame.append(rmse_direct, ignore_index=True)
        rmse_frame.index = ["Estimated", "Girth", "Initial", "direct"]
    else:
        rmse_frame.index = ["Estimated", "Girth", "Initial"]
    print(rmse_frame.to_markdown())

                                                                                

In [8]:
performance_dict = mirt_performance_benchmark()

EM Iteration 1
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 1: current parameter_diff: 1.4433343094270454, current marginal loglikelihood: -375.59868262627896
EM Iteration 2
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 2: current parameter_diff: 1.3774079583421837, current marginal loglikelihood: -374.0318973595513
EM Iteration 3
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 3: current parameter_diff: 1.3220300604197965, current marginal loglikelihood: -373.21558088186475
EM Iteration 4
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 4: current parameter_diff: 0.05989541760402883, current marginal loglikelihood: -373.02296365493294
Absolute diff in A:
[[0.22726411 0.        ]
 [0.68911161 0.        ]
 [0.         0.94531536]
 [0.3        0.4       ]
 [0.83936821 0.68840146]
 [0.         0.38979184]]
Absolute diff in delta:
[0.3033585  0.18678488 0.03817952 0.2006707  0.49599371 0.18184322]
Absolute diff in sigma:
[[0.         0.12064202]
 [0.12064202 0.        ]

c:\Users\Jesper\Anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: function values with index [11] are not finite but [inf]. (class=CMAEvolutionStrategy method=ask iteration=2)
  warnings.warn(msg + ' (' +
c:\Users\Jesper\Anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: function values with index [11] are not finite but [inf]. (class=CMAEvolutionStrategy method=ask iteration=4)
  warnings.warn(msg + ' (' +
c:\Users\Jesper\Anaconda3\lib\site-packages\cma\utilities\utils.py:343: UserWarning: function values with index [0] are not finite but [inf]. (class=CMAEvolutionStrategy method=ask iteration=11)
  warnings.warn(msg + ' (' +


   47    564 3.722559433274413e+02 2.2e+00 7.74e-02  5e-02  9e-02 0:03.3
  100   1200 3.719973934408869e+02 3.3e+00 2.99e-02  2e-02  3e-02 0:06.8
  175   2100 3.719754400919931e+02 5.2e+00 1.87e-02  9e-03  2e-02 0:11.9
  200   2400 3.719552722854544e+02 5.8e+00 2.77e-02  2e-02  3e-02 0:13.6
  300   3600 3.719383523543403e+02 9.7e+00 1.15e-02  6e-03  2e-02 0:20.3
  400   4800 3.720002646044438e+02 1.2e+01 4.82e-03  2e-03  6e-03 0:27.0
  500   6000 3.719659617699999e+02 1.9e+01 3.85e-03  2e-03  4e-03 0:33.6
  600   7200 3.720050161057476e+02 2.0e+01 2.04e-03  7e-04  2e-03 0:40.3
  700   8400 3.719155106530180e+02 3.5e+01 1.08e-03  4e-04  1e-03 0:47.0
  765   9180 3.719565294316205e+02 4.6e+01 1.82e-03  7e-04  2e-03 0:51.3
Absolute diff in A:
[[0.01173228 0.        ]
 [0.59011674 0.        ]
 [0.         1.19667274]
 [0.58093851 0.06351766]
 [0.79122824 0.66038916]
 [0.         0.88650999]]
Absolute diff in delta:
[0.28490015 0.1407527  0.11259298 0.29793302 0.59783111 0.3378697 ]
Absolut

In [19]:
print_result(performance_dict, "Monte Cralo with fixed sample per em-step")

------------------------------------
##### Results for Monte Cralo with fixed sample per em-step
Runtime: 1.22 seconds, 5 steps, 0.24 seconds per step \
Optimal marginal Likelihood: -381.06, Estimated: -373.26, Initial -377.69
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.500043 |     0.273166 |    0.0853068 |
| Girth     | 0.98542  |     0.334766 |    0.141421  |
| Initial   | 0.509902 |     0.431741 |    0.141421  |
| direct    | 0.57518  |     0.335156 |    0.0914744 |


------------------------------------
##### Results for Naive, all-Genetic, no vectorization
Runtime: 29100 seconds, 100+ steps, 291 seconds per step\
Performance: rmse-mean = 0.5896464054719196

------------------------------------
##### Results for Vectorized q_item
Runtime: 613.3871161937714 seconds, 100+ steps, 6.0731397642947655 seconds per step \
Performance: rmse-mean = 0.6466789259215839

------------------------------------
##### Results for More precise results through higher N and pop_size
Runtime: 2239.073846578598 seconds, 100+ steps, 22.169047985926714 seconds per step \
Performance: rmse-mean = 0.6384660709857508

------------------------------------
##### Results for Debug GA sorting, Add Likelihood-based stopping criterion
Runtime: 135.11203932762146 seconds, 7 steps, 19.301719903945923 seconds per step \
Performance: rmse-mean = 0.5616448369465917

------------------------------------
##### Results for Add vectorization in Q_0
Runtime: 228.78 seconds, 17 steps, 13.46 seconds per step \
Performance: rmse-mean = 0.6174

------------------------------------
##### Results for Add vectorization in E-Step normalizing constant
Runtime: 28.37 seconds, 9 steps, 3.15 seconds per step \
Performance: rmse-mean = 0.5808

------------------------------------
##### Results for Add Suggested Initial Parameters (Zhang)
Runtime: 61.22 seconds, 19 steps, 3.22 seconds per step \
Performance: rmse-mean = 0.5625

------------------------------------
##### Results for Fixed Q_item
Runtime: 81.11 seconds, 12 steps, 6.76 seconds per step \
Optimal marginal Likelihood: -375.43, Estimated: -393.8, Initial -391.83
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.762005 |     0.512703 |     0.600404 |
| Girth     | 1.61804  |     0.750525 |   nan        |
| Initial   | 0.770281 |     0.227528 |     0.141421 |

------------------------------------
##### Results for Inproved GA stoppimg rule
Runtime: 42.54 seconds, 7 steps, 6.08 seconds per step \
Optimal marginal Likelihood: -378.68, Estimated: -391.87, Initial -378.08
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.845078 |     0.568092 |    0.167374  |
| Girth     | 0.930362 |     0.554795 |    0.141421  |
| direct    | 0.401578 |     0.609516 |    0.0526782 |
| Initial   | 0.509902 |     0.364352 |    0.141421  |

------------------------------------
##### Results for pycma ga used
Runtime: 489.8 seconds, 16 steps, 30.61 seconds per step \
Optimal marginal Likelihood: -381.34, Estimated: -414.28, Initial -379.77
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.942127 |     0.505808 |     0.215139 |
| Girth     | 1.42144  |     0.632431 |   nan        |
| Initial   | 0.509902 |     0.489947 |     0.141421 |

------------------------------------
##### Results for pycma, higher sigma0
Runtime: 282.36 seconds, 10 steps, 28.24 seconds per step \
Optimal marginal Likelihood: -368.93, Estimated: -399.56, Initial -369.53
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.898735 |     0.48221  |     0.195257 |
| Girth     | 1.79316  |     0.515695 |   nan        |
| direct    | 0.458306 |     0.262779 |     0.139413 |
| Initial   | 0.509902 |     0.273981 |     0.141421 |

------------------------------------
##### Results for Monte Carlo with seed
Runtime: 97.6 seconds, 9 steps, 10.84 seconds per step \
Optimal marginal Likelihood: -376.79, Estimated: -399.07, Initial -372.28
|           |      rmse_A |   rmse_delta |   rmse_sigma |
|:----------|------------:|-------------:|-------------:|
| Estimated |    1.09615  |     0.970204 |     0.120425 |
| Girth     |    1.45372  |     0.34729  |     0.141421 |
| direct    | 2935.78     |  1035.68     |     0.014461 |
| Initial   |    0.509902 |     0.445143 |     0.141421 |

------------------------------------
##### Results for Monte Cralo with fixed sample per em-step
Runtime: 1.22 seconds, 5 steps, 0.24 seconds per step \
Optimal marginal Likelihood: -381.06, Estimated: -373.26, Initial -377.69
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.500043 |     0.273166 |    0.0853068 |
| Girth     | 0.98542  |     0.334766 |    0.141421  |
| direct    | 0.57518  |     0.335156 |    0.0914744 |
| Initial   | 0.509902 |     0.431741 |    0.141421  |

## Procedure for better runtime

1. Add Performance Benchmark (done)
2. Add Vectorization to q_item (done)
3. Add better Initialization (done)
3. Add Vectorization to q_0 (done)
4. Add q_0 Derivation 1. + BFGS
5. Add q_0 Derivation 2. + Newton Raphson
6. Use cython (maybe)

## Experiment 1: MIRT-2PL Parameter Recovery

In [27]:
from unittest import result


def mirt_param_recovery(sample_size, item_dimension = 20, latent_dimension=3, q_type="seperated") -> dict:
    
    #Define Population
    population = respondent_population(latent_dimension=latent_dimension)
    real_latent_cov = population.initialize_random_person_parameters()

    #Sample responses
    response_simulation_obj = response_simulation(population=population, item_dimension=item_dimension)
    response_simulation_obj.initialize_random_q_structured_matrix(structure=q_type)
    early_item_parameters = response_simulation_obj.initialize_random_item_parameters()
    early_item_parameters.update({"item_dimension": item_dimension, "latent_dimension": latent_dimension})
    sample = response_simulation_obj.sample(sample_size=sample_size)
    real_early_parameters = {"item_parameters": early_item_parameters, "person_parameters": {"covariance": real_latent_cov}}

    #Fit Parameters
    #Initialize model
    model = models.mirt_2pl(latent_dimension=latent_dimension, item_dimension=item_dimension, Q=early_item_parameters["q_matrix"])
    model.initialize_from_responses(response_data=sample["early_responses"])
    initial_early_parameters = model.get_parameters()
    e_step = em_algorithm.e_step_ga_mml(model=model)
    m_step = em_algorithm.m_step_ga_mml(model)
    em = em_algorithm.em_algo(e_step=e_step, m_step=m_step, model=model)

    #Fit Model
    start_time = time.time()
    em.fit(sample["early_responses"], max_iter=50)
    run_time =  (time.time() - start_time)

    #Measure Performance
    early_estimated_item_parameters = em.model.item_parameters
    early_estimated_person_parameters = em.model.person_parameters

    #Create Baselines
    baselines = {"early_initial": {"parameters": initial_early_parameters}, "girth": {}}
    
    #Create results
    early_estimated_parameters = em.model.get_parameters()

    parameter_dict = create_parameter_dict(estimated_early_parameters=early_estimated_parameters,
                          real_early_parameters = real_early_parameters,
                          estimated_late_parameters=None, real_late_parameters=None)
    run_dict = {"early": {"runtime": run_time,
                "number_steps": em.n_steps}}
    performance_dict = create_performance_dict(parameter_dict=parameter_dict, run_dict=run_dict, sample=sample, baselines=baselines, model=model)

    return(performance_dict)

In [28]:
#Idee, Delta so initialisieren, dass die relativen Itemschwierigkeiten um 0 herum verteilt sind!
result_dict = mirt_param_recovery(1000, q_type="singular", item_dimension=20, latent_dimension=1)

EM Iteration 1
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 1: current parameter_diff: 19.150981026575828, current marginal loglikelihood: -10262.671183347536
EM Iteration 2
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 2: current parameter_diff: 11.954677283140864, current marginal loglikelihood: -10110.49209658875
EM Iteration 3
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 3: current parameter_diff: 6.825496600674044, current marginal loglikelihood: -10084.534120657478
EM Iteration 4
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 4: current parameter_diff: 4.8035381163832, current marginal loglikelihood: -10066.272249423451
EM Iteration 5
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 5: current parameter_diff: 3.7971986925751984, current marginal loglikelihood: -10059.41325962015
EM Iteration 6
E-step
M-step
Maximize Q-0
Maximize the Q_i's
Step: 6: current parameter_diff: 3.215132074642579, current marginal loglikelihood: -10059.707701098803
EM Iteratio

In [22]:
print_result(result_dict, "Fixed sample per em-step")

------------------------------------
##### Results for Fixed sample per em-step
Runtime: 3.79 seconds, 6 steps, 0.63 seconds per step \
Optimal marginal Likelihood: -1078.73, Estimated: -1062.7, Initial -1111.35
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.574371 |     0.388029 |            0 |
| Girth     | 0.957604 |     4.42778  |            0 |
| Initial   | 1.05762  |     0.936002 |            0 |


### Univariate IRT/Full GA

------------------------------------
##### Item Dimension=12, sample_size=200: Results for Fixed sample per em-step
Runtime: 4.38 seconds, 7 steps, 0.63 seconds per step \
Optimal marginal Likelihood: -1284.1, Estimated: -1273.09, Initial -1330.43
|           |   rmse_A |   rmse_delta |   rmse_sigma |
|:----------|---------:|-------------:|-------------:|
| Estimated | 0.508687 |     0.196944 |            0 |
| Girth     | 0.377882 |     2.65875  |            0 |
| direct    | 0.305489 |     0.318168 |            0 |
| Initial   | 1.5583   |     0.861273 |            0 |

### Procedure for better model performance:

1. Implement MIRT Baseline (done)
2. Test GA with quadratic function (done)
3. use pycma (done)
4. Test mirt-functions (response-matrix-probability, done)
5. Read on MC MIRT (MC-Errorbounds usage or quasi-MC or importance sampling)
6. Use importance sampling
7. Implement Newton Raphson

### Estimeted Parameters